In [ ]:
import numpy as np
import os
import rasterio as rio

In [ ]:
class Box:
    def __init__(self, bounds=None, left:int=0, bottom:int=0, right:int=0, top:int=0):
        if bounds is not None: self.left,self.bottom,self.right,self.top = bounds
        else: self.left,self.bottom,self.right,self.top = left,bottom,right,top
        self.width, self.height = self.right -self.left, self.top -self.bottom
    def __str__(self):        
        return "left:{} bottom:{} right:{} top:{} width:{} height:{}".format(
            self.left, self.bottom, 
            self.right, self.top, 
            self.width, self.height)
    def in_bounds(self, x:int, y:int) -> bool:
        return self.left < x < self.right and self.bottom < y < self.top


def cropper(origin, new_name, x, y, new_width, new_height):
        origin_arr = np.array(origin.read(1))
        origin_bounds = origin.bounds
      
        left = int(x * width)
        right = int((x+1) * width)
        bottom = int((y+1) * height)
        top = int(y * height)
            
        new_left = origin_bounds.left + left
        new_right = origin_bounds.left + right
        new_bottom = origin_bounds.top - bottom
        new_top = origin_bounds.top - top
        
        print(f"[{x},{y}]: {new_left}{new_right}|{new_bottom}{new_top}")
        # crop out of the array
        new_arr = origin_arr[left:right,top:bottom]
        
        new_meta = origin.meta
        
        # transform needs to be edited so the crop has correct coords
        new_meta['transform'](
            # new left-most coord
            new_left,
            # new bottom-most coord
            new_bottom,
            # new right-most coord
            new_right,
            # new top-most coord
            new_top,
            # new width
            new_width,
            # new height
            new_height)

        new_meta["width"] = new_width
        new_meta["height"] = new_height

        new_path = os.path.join("/media/seppe/DiskSpaceInvader/3DHouseData",new_name)
        # open a new file in write mode as destination
        # **meta is the **kwargs passed to the tif
        with rio.open(new_path, "w", **new_meta) as destination:
            # write the pixel data with the **meta data
            destination.write(new_arr, indexes=1)
            print("new file: ",new_name)
            destination.close()
        return